In [1]:
import requests

api_key = "ragflow-YyYWRhZmY0ODMyMTExZjA4OTQ3MDI0Mm"
base_url = "http://10.50.3.11:80"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
def create_session(chat_id, session_name=None):
    """새 세션을 생성하는 함수"""
    url = f"{base_url}/api/v1/chats/{chat_id}/sessions"
    data = {
        "name": session_name
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        session_id = response.json()["data"]["id"]
        print(f"Created new session: {session_id}")
        return session_id
    else:
        print(f"Error creating session: {response.status_code} - {response.text}")        return None

In [ ]:
import json
def get_chat_by_name(chat_name):
    """chat name으로 chat id를 구하는 함수"""
    url = f"{base_url}/api/v1/chats"
    params = {
        "page": 1,
        "page_size": 100,
        "name": chat_name
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("data") and len(data["data"]) > 0:
            chat = data["data"][0]
            # json data를 indent하여 출력
            print(json.dumps(chat, indent=2, ensure_ascii=False))
            print(f"Found chat '{chat_name}' with ID: {chat['id']}")
            return chat
        else:
            print(f"No chat found with name: {chat_name}")
            return None
    else:
        print(f"Error getting chats: {response.status_code} - {response.text}")
        return None

In [54]:
import json
chat = get_chat_by_name("흥국화재_사내문서")
#print(f"chat: {json.dumps(chat, indent=2, ensure_ascii=False)}")
print(f'prompt: {chat["prompt"]["prompt"]}')

Found chat '흥국화재_사내문서' with ID: 12b0d8ec78b711f0849a0242ac1e0006
prompt: You are an intelligent assistant. Please summarize the content of the knowledge base to answer the question. Please list the data in the knowledge base and answer in detail. When all knowledge base content is irrelevant to the question, your answer must include the sentence "The answer you are looking for is not found in the knowledge base!" Answers need to consider chat history. Answers should list all citations which is used to generate answers.
      Here is the knowledge base:
      {knowledge}
      The above is the knowledge base.


In [ ]:
def update_chat_prompt(chat_name, **kwargs):
    """chat id로 chat을 업데이트하는 함수"""
    # get the prompt from the chat (existing template may exist)
    chat = get_chat_by_name(chat_name)
    if not chat:
        print(f"Chat not found: {chat_name}")
        return None

    # new chat prompt to update
    prompt_template = """
You are an intelligent assistant. Please follow the user's instructions and use the knowledge base to produce concrete, citation-backed modifications to the current sales script. When all knowledge base content is irrelevant to the question, your answer must include the sentence "The answer you are looking for is not found in the knowledge base!" Answers need to consider chat history. Answers should list all citations which are used to generate answers.
Task for the assistant:
- Keep the original format of the current sales script
- Refer to the newly retrieved knowledge
- Follow the instructions provided in user_feedback
- Generate a new sales script

    Here is the user feedback (instructions provided by the user):
    {user_feedback}
    The above is the user feedback.

    Here is the knowledge base:
    {knowledge}
    The above is the knowledge base.

    Here is the current sales script:
    {current_script}
    The above is the current sales script.
"""

    user_feedback = kwargs.get("user_feedback", "")
    current_script = kwargs.get("current_script", "")

    new_prompt = prompt_template  # default to the raw template
    # Use placeholders for knowledge by default; embed provided user_feedback/current_script
    filled = {
        'knowledge': '{knowledge}',
        'user_feedback': user_feedback if user_feedback else '',
        'current_script': current_script if current_script else ''
    }
    try:
        new_prompt = prompt_template.format(**filled)
    except Exception:
        # fallback to the raw template if formatting fails
        new_prompt = prompt_template

    # Prepare payload and stest_chat_idend update
    chat_id = chat.get("id")
    url = f"{base_url}/api/v1/chats/{chat_id}"

    # Build prompt_data from chat prompt safely and update required fields
    prompt_data = chat.get("prompt", {}).copy()
    prompt_data.update({
        "prompt": new_prompt,
        "variables": [
            {"key": "knowledge", "optional": False},
            {"key": "user_feedback", "optional": True},
            {"key": "current_script", "optional": True},
        ],
    })

    data = {
        "prompt": prompt_data,
        "name": chat.get("name"),
    }

    response = requests.put(url, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Successfully updated chat {chat_id}")
        return response.json()
    else:
        print(f"Error updating chat: {response.status_code} - {response.text}")
        return None



In [56]:
# --- Test block for update_chat_prompt ---
# This will lookup the chat by name, then call update_chat_prompt with sample inputs.
import json

chat_name = "테스트"
if not chat:
    print(f"Test aborted: chat '{chat_name}' not found.")
else:
    sample_user_feedback = """
무배당 흥Good 행복든든 재산보험(25.06)_(1종) 상품의 장점을 더 부각시키고, 고객이 쉽게 이해할 수 있도록 설명을 간결하게 해주세요. 
또한, 고객이 보험 가입 후 빠른 보상 처리를 받을 수 있다는 점을 강조해 주세요.
스프링클러가 없는 건물이나 아파트가 많은데 이거에 대한 보장도 가능한지 여부를 추가로 설명해 주세요.
"""
    sample_current_script = """
[인사말]
안녕하세요, 고객님. 요즘 집안에서 화재나 누수 같은 작은 사고라도 생기면 복구 비용이 만만치 않다는 걸 많이들 느끼셨을 텐데요. 예전 같으면 이런 부분은 그냥 감수하시거나 보험 없이 넘어가셨을 텐데, 요즘은 보다 체계적으로 대비하는 분들이 많아졌죠.
생활하면서 예상치 못한 손해는 누구에게나 찾아올 수 있고, 특히 집에 대한 보장은 꼭 필요합니다. 특히 40대 분들 같은 경우, 가족과 집을 동시에 안정적으로 보호하고 싶어 하시는 경우가 많아요.
오늘은 그런 부분을 좀 더 안심하고 대비할 수 있는 방법에 대해 간단히 안내드리고자 연락드린 건데, 혹시 관심 있으신가요?
[보험 영업]
오늘 안내들리 상품은 **무배당 흥Good 행복든든 재산보험(25.06)_(1종)**입니다. 이 상품은 집에 대한 다양한 손해를 한 번에 보장해 드리는 종합 보험으로, 특히 화재나 물 누수 같은 사고에 대해 실손을 보상해 드리고 있어요.
주요 보장은 크게 네가지로 구성되어 있는데요. 첫 번째는 **화재손해(건물)(실손보상형)**입니다. 집이 화재로 인해 손상되었을 때 실제 지출한 복구 비용을 보상해 드리고요.
두 번째는 **스프링클러 누출손해(비례보상형)**입니다. 집에 설치된 스프링클러가 고장이나 누수로 인해 손해가 발생했을 때, 보상 한도 내에서 비례해서 보상해 드리는 부분이에요.
또한, **임대인배상책임(주택)(화재배상제외)**과 **화재배상책임**도 포함되어 있어요, 이 부분은 집을 임대하거나 거주하면서 발생할 수 있는 제 3자 배상 책임도 커버해 드리고요.
특히 고객님처럼 40대 중반에 계시고, 집을 소유하고 계신 분들께는 이런 종합적인 보장이 큰 도움이 될 수 있어요. 집은 단 한 번의 사고로도 큰 손해가 발생할 수 있기 때문에, 미리 대비해 두는게 중요하죠.
이 상품은 집에 대한 보장을 체계적으로 구성할 수 있도록 설계되어 있어서, 보험에 익숙하지 않으신 분들도 쉽게 이해하시고 가입하실 수 있어요.
[마무리]
월 1만 원, 7년 납입하는 상품으로, 집에 대한 다양한 손해를 보장받을 수 있어요. 화재나 물 누수뿐 아니라 배상책임도 함께 커버해 드리고요.
이 정도 금액으로 집의 안전을 체계적으로 지켜볼 수 있다면, 꼭 고려해보시는 게 좋을 것 같아요.
궁금한 점이나 문의하실 내용 있으시면 언제든지 말씀해 주세요. 도움을 드릴 수 있어 정말 기쁩니다.
[요약]
고객님처럼 집을 소유하고 계신 분께는 예상치 못한 화재나 누수에 대비하는 것이 중요합니다.
주요 보장은 화재손해(건물)(실손보상형), 스프링클러 누출손해(비례보상형), 임대인배상 책임, 화재배상착임입니다.
집에 대한 다양한 손해를 한 번에 보장받을 수 있어, 가입 시 큰 도움이 될 수 있습니다.
"""

    print("사용자 피드백과 현재 스크립트를 기반으로 update_chat_prompt() 호출")
    result = update_chat_prompt(chat_name, user_feedback=sample_user_feedback, current_script=sample_current_script)
    print("Result:")
    try:
        print(json.dumps(result, ensure_ascii=False, indent=0))
    except Exception:
        print(result)

사용자 피드백과 현재 스크립트를 기반으로 update_chat_prompt() 호출
Found chat '테스트' with ID: 7ff68e5883c811f089040242c0a84006
Successfully updated chat 7ff68e5883c811f089040242c0a84006
Result:
{
"code": 0
}
Successfully updated chat 7ff68e5883c811f089040242c0a84006
Result:
{
"code": 0
}
